<a href="https://colab.research.google.com/github/younghoonNa/pythonDT/blob/main/%E1%84%83%E1%85%A6%E1%84%8B%E1%85%B5%E1%84%90%E1%85%A5%E1%84%87%E1%85%AE%E1%86%AB%E1%84%89%E1%85%A5%E1%86%A8_5%E1%84%8B%E1%85%B5%E1%86%AF%E1%84%8E%E1%85%A1_%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%85%E1%85%A7%E1%86%A8%E1%84%89%E1%85%A1%E1%84%8B%E1%85%AD%E1%86%BC%E1%84%85%E1%85%A3%E1%86%BC_%E1%84%8B%E1%85%A8%E1%84%8E%E1%85%B3%E1%86%A8_%E1%84%8B%E1%85%AF%E1%84%8F%E1%85%B3%E1%84%89%E1%85%A3%E1%86%B822.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정

In [1]:
!wget http://49.247.133.7/ws2.zip
!unzip ws2.zip

--2021-12-31 08:15:38--  http://49.247.133.7/ws2.zip
Connecting to 49.247.133.7:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1452903 (1.4M) [application/zip]
Saving to: ‘ws2.zip’

ws2.zip             100%[===================>]   1.38M  1.61MB/s    in 0.9s    

2021-12-31 08:15:39 (1.61 MB/s) - ‘ws2.zip’ saved [1452903/1452903]

Archive:  ws2.zip
  inflating: submission.csv          
  inflating: test.csv                
  inflating: train.csv               


In [2]:
!pip install jaen

In [3]:
# 실전프로젝트 환경 설정
from JAEN.project import Project
pjt = Project('전력 사용량 예측',
              '한림대-데이터분석과정_전공자',                    
              '1차수 A반',                    
              'yh07280@naver.com')             

# 모듈 import

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

# 데이터 로드
2개의 데이터 셋트를 나누어 제공합니다.

1. train: 학습용 데이터 세트. 정답 값이라고 할 수 있는 `label` 값이 같이 제공됩니다.
2. test: 예측용 데이터 세트. 정답 값이 빠진 `feature` 들만 제공됩니다. 우리는 이 `feature`를 활용하여 예측을 진행합니다.
3. submission: 정답 제출용 데이터 세트. 주어진 형식에 맞춰 정답 값을 기입하고 제출합니다.|

In [5]:
# train 데이터셋 로드 (train.csv)
train = pd.read_csv('train.csv')
train
# 60개 건물 X 61일 X 24시간 = 87840

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
87835,60,2020-07-31 19:00:00,3603.744,25.0,2.7,95.0,0.0,0.0,1.0,1.0
87836,60,2020-07-31 20:00:00,3688.848,24.9,2.5,96.0,0.0,0.0,1.0,1.0
87837,60,2020-07-31 21:00:00,3765.312,24.9,1.9,96.0,0.0,0.0,1.0,1.0
87838,60,2020-07-31 22:00:00,3833.568,24.9,2.8,97.0,0.0,0.0,1.0,1.0


In [62]:
# test 데이터셋 로드 (test.csv)
test = pd.read_csv('test.csv')
test

# 60개 건물 X 24일 X 24시간 = 34560

,num,date_time,기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-08-01 00:00:00,25.0,3.4,90.0,1.0,0.0,0.0,0.0
1,1,2020-08-01 01:00:00,24.6,0.5,90.0,0.2,0.0,0.0,0.0
2,1,2020-08-01 02:00:00,24.6,0.5,91.0,0.0,0.0,0.0,0.0
3,1,2020-08-01 03:00:00,24.7,1.2,93.0,0.0,0.0,0.0,0.0
4,1,2020-08-01 04:00:00,24.7,0.8,94.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
34555,60,2020-08-24 19:00:00,27.8,2.3,68.0,0.0,0.7,1.0,1.0
34556,60,2020-08-24 20:00:00,27.3,1.2,71.0,0.0,0.0,1.0,1.0
34557,60,2020-08-24 21:00:00,27.3,1.8,71.0,0.0,0.0,1.0,1.0
34558,60,2020-08-24 22:00:00,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [7]:
# train column 별 데이터 개수, 타입 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87840 entries, 0 to 87839
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   num         87840 non-null  int64  
 1   date_time   87840 non-null  object 
 2   전력사용량(kWh)  87840 non-null  float64
 3   기온(°C)      87840 non-null  float64
 4   풍속(m/s)     87840 non-null  float64
 5   습도(%)       87840 non-null  float64
 6   강수량(mm)     87840 non-null  float64
 7   일조(hr)      87840 non-null  float64
 8   비전기냉방설비운영   87840 non-null  float64
 9   태양광보유       87840 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 6.7+ MB


In [8]:
# test column 별 데이터 개수, 타입 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34560 entries, 0 to 34559
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   num        34560 non-null  int64  
 1   date_time  34560 non-null  object 
 2   기온(°C)     34560 non-null  float64
 3   풍속(m/s)    34560 non-null  float64
 4   습도(%)      34560 non-null  float64
 5   강수량(mm)    34560 non-null  float64
 6   일조(hr)     34560 non-null  float64
 7   비전기냉방설비운영  34560 non-null  float64
 8   태양광보유      34560 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 2.4+ MB


In [9]:
# train 결측치 확인 
train.isna().sum(axis=0)

num           0
date_time     0
전력사용량(kWh)    0
기온(°C)        0
풍속(m/s)       0
습도(%)         0
강수량(mm)       0
일조(hr)        0
비전기냉방설비운영     0
태양광보유         0
dtype: int64

In [10]:
# test 결측치 확인
test.isna().sum(axis=0)

num          0
date_time    0
기온(°C)       0
풍속(m/s)      0
습도(%)        0
강수량(mm)      0
일조(hr)       0
비전기냉방설비운영    0
태양광보유        0
dtype: int64

In [11]:
# 본인의 판단에 따라 훈련에 사용할 feature와 사용하지 않을 feature를 선택합니다.
features = [
    'num',
    '기온(°C)',
    '풍속(m/s)',
    '습도(%)',
    '강수량(mm)',
    '일조(hr)',
    '비전기냉방설비운영',
    '태양광보유',
]
features

['num',
 '기온(°C)',
 '풍속(m/s)',
 '습도(%)',
 '강수량(mm)',
 '일조(hr)',
 '비전기냉방설비운영',
 '태양광보유']

In [58]:
# X = train.drop(['전력사용량(kWh)', 'date_time'], axis = 1)
X = train[features]
# features의 컬럼을 X로 설정한다.
X

,num,기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
87835,60,25.0,2.7,95.0,0.0,0.0,1.0,1.0
87836,60,24.9,2.5,96.0,0.0,0.0,1.0,1.0
87837,60,24.9,1.9,96.0,0.0,0.0,1.0,1.0
87838,60,24.9,2.8,97.0,0.0,0.0,1.0,1.0


In [14]:
# 전력사용량(kWh) 값을 Y값으로 설정한다.
y = train['전력사용량(kWh)']
y

0        8179.056
1        8135.640
2        8107.128
3        8048.808
4        8043.624
           ...   
87835    3603.744
87836    3688.848
87837    3765.312
87838    3833.568
87839    3309.120
Name: 전력사용량(kWh), Length: 87840, dtype: float64

In [ ]:
# 표준화 적용
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # 변환기 생성
scaler.fit(X) # 각 컬럼의 평균과 표준편차를 알아내는 작업
x_scaled = scaler.transform(X) # 표준화 적용
pd.DataFrame(x_scaled).describe().round(2)

In [15]:
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.1)

In [30]:
# RandomForestRegressor 사용
  # n_estimators=500,random_state=123적용
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=10,random_state=23).fit(x_train, y_train)
model.score(x_train, y_train), model.score(x_test, y_test)

(0.8649968887575398, 0.8508758788567085)

In [51]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators=1000,random_state=23)
model.fit(x_train, y_train)

[08:24:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(n_estimators=1000, random_state=23)

In [55]:
# svm_confirmed = svm_search.best_estimator_
from sklearn.svm import SVR
svm_rbf_clf =  SVR(kernel="rbf", gamma = 10, C = 100, epsilon=0.01)
svm_rbf_clf.fit(x_train, y_train)

SVR(C=100, epsilon=0.01, gamma=10)

In [40]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=5)
model.fit(x_train, y_train)

KNeighborsRegressor()

In [56]:
model.score(x_train, y_train), model.score(x_test, y_test)

(0.8784390288159912, 0.8691922124254559)

In [32]:
test

,num,기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,25.0,3.4,90.0,1.0,0.0,0.0,0.0
1,1,24.6,0.5,90.0,0.2,0.0,0.0,0.0
2,1,24.6,0.5,91.0,0.0,0.0,0.0,0.0
3,1,24.7,1.2,93.0,0.0,0.0,0.0,0.0
4,1,24.7,0.8,94.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
34555,60,27.8,2.3,68.0,0.0,0.7,1.0,1.0
34556,60,27.3,1.2,71.0,0.0,0.0,1.0,1.0
34557,60,27.3,1.8,71.0,0.0,0.0,1.0,1.0
34558,60,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [33]:
test_x = test.drop(['date_time'], axis = 1)
test_x

KeyError: ignored

In [52]:
# test 데이터셋에 대한 예측
fin = model.predict(test_x)
fin

array([8314.219 , 8718.772 , 8649.264 , ..., 3544.191 , 3588.4988,
       3593.99  ], dtype=float32)

In [53]:
# submission 데이터 로드 (submission.csv)
submission = pd.read_csv('submission.csv')
submission['전력사용량(kWh)'] = fin
# 예측 결과 대입
submission

,num,date_time,전력사용량(kWh)
0,1,2020-08-01 00:00:00,8314.218750
1,1,2020-08-01 01:00:00,8718.772461
2,1,2020-08-01 02:00:00,8649.263672
3,1,2020-08-01 03:00:00,8632.441406
4,1,2020-08-01 04:00:00,8651.513672
...,...,...,...
34555,60,2020-08-24 19:00:00,3810.382080
34556,60,2020-08-24 20:00:00,3460.931152
34557,60,2020-08-24 21:00:00,3544.190918
34558,60,2020-08-24 22:00:00,3588.498779


In [ ]:
print('MAE:', mean_absolute_error(svm_rbf_pred, submission))
print('MSE:',mean_squared_error(svm_rbf_pred, y_test_price))

# 데이터 전처리, 모델 튜닝 등 성능을 더 높여보세요
- 제출 결과가 낮을수록 좋은 모델입니다.

In [54]:
# 결과 제출 
pjt.submit(submission)

파일을 저장하였습니다. 파일명: submission-08-24-49.csv


KeyError: ignored

In [ ]:
from sklearn.metrics import mean_squared_log_error
P = model.predict(X)
mean_squared_log_error(Y, P, squared=False)

In [ ]:
# 본인의 판단에 따라 훈련에 사용할 feature와 사용하지 않을 feature를 선택합니다.
features = [
    'num',
    '기온(°C)',
    '풍속(m/s)',
    '습도(%)',
    '강수량(mm)',
    '일조(hr)',
    '비전기냉방설비운영',
    '태양광보유',
    'month',
    'weekday',
    'hour'
]

X = train[features]
model = RandomForestRegressor(n_estimators=100, random_state=123).fit(X, Y)
pred = model.predict(test[features])
submission['전력사용량(kWh)'] = pred
pjt.submit(submission) # RMSLE

In [59]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
87835,60,2020-07-31 19:00:00,3603.744,25.0,2.7,95.0,0.0,0.0,1.0,1.0
87836,60,2020-07-31 20:00:00,3688.848,24.9,2.5,96.0,0.0,0.0,1.0,1.0
87837,60,2020-07-31 21:00:00,3765.312,24.9,1.9,96.0,0.0,0.0,1.0,1.0
87838,60,2020-07-31 22:00:00,3833.568,24.9,2.8,97.0,0.0,0.0,1.0,1.0


In [60]:
train['date_time'] = pd.to_datetime(train['date_time'])
train['date_time']

0       2020-06-01 00:00:00
1       2020-06-01 01:00:00
2       2020-06-01 02:00:00
3       2020-06-01 03:00:00
4       2020-06-01 04:00:00
                ...        
87835   2020-07-31 19:00:00
87836   2020-07-31 20:00:00
87837   2020-07-31 21:00:00
87838   2020-07-31 22:00:00
87839   2020-07-31 23:00:00
Name: date_time, Length: 87840, dtype: datetime64[ns]

In [63]:
train['date_time'] = pd.to_datetime(train['date_time'])
train['month'] = train['date_time'].dt.month
train['weekday'] = train['date_time'].dt.weekday
train['hour'] = train['date_time'].dt.hour

test['date_time'] = pd.to_datetime(test['date_time'])
test['month'] = test['date_time'].dt.month
test['weekday'] = test['date_time'].dt.weekday
test['hour'] = test['date_time'].dt.hour

AttributeError: ignored